In [1]:
import requests
import json
import pandas as pd
import numpy as np
import acquire
import datetime as dt

In [2]:
df = pd.read_csv('combined.csv')

In [3]:
df.head()

,Unnamed: 0,item_id,sale_amount,sale_date,sale_id,store_id,store_address,store_city,store_state,store_zipcode,item_brand,item_name,item_price,item_upc12,item_upc14
0,0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
2,2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
3,3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
4,4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013


Write a function to convert a date to adatetime data type.

Write a function to change a datetime to UTC.

In [4]:
df['sale_date'] = pd.to_datetime(df['sale_date'], utc=True)

Write a function to parse a date column into 6 additional columns (while keeping the original date): year, quarter, month, day of month, day of week, weekend vs. weekday

In [5]:
def split_date_series(series: pd.Series) -> tuple:
    """
    Return six columns: year, quarter, month, day, weekday #, and whether it's a weekday (1) or weekend (0)
    """
    year = series.dt.year.rename("year")
    quarter = series.dt.quarter.rename("quarter")
    month = series.dt.month.rename("month")
    day = series.dt.day.rename("day")
    weekday = series.dt.weekday.rename("weekday")
    is_weekday = (series.dt.weekday < 5).rename("is_weekday")
    return year, quarter, month, day, weekday, is_weekday

In [6]:
# split_date_series(df['sale_date'])

In [7]:
df = pd.concat([df, *split_date_series(df.sale_date)], axis=1)

In [8]:
df.head()

,Unnamed: 0,item_id,sale_amount,sale_date,sale_id,store_id,store_address,store_city,store_state,store_zipcode,...,item_name,item_price,item_upc12,item_upc14,year,quarter,month,day,weekday,is_weekday
0,0,1,13.0,2013-01-01 00:00:00+00:00,1,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,1,1,True
1,1,1,11.0,2013-01-02 00:00:00+00:00,2,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,2,2,True
2,2,1,14.0,2013-01-03 00:00:00+00:00,3,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,3,3,True
3,3,1,13.0,2013-01-04 00:00:00+00:00,4,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,4,4,True
4,4,1,10.0,2013-01-05 00:00:00+00:00,5,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,Riceland American Jazmine Rice,0.84,35200264013,35200264013,2013,1,1,5,5,False


Add a column to your dataframe, sales_total, which is a derived from sale_amount (total items) and item_price.

In [9]:
df.rename(columns = {'sale_amount': 'quantity'}, inplace=True)

In [21]:
df.head()

,Unnamed: 0,item_id,quantity,sale_date,sale_id,store_id,store_address,store_city,store_state,store_zipcode,...,item_price,item_upc12,item_upc14,year,quarter,month,day,weekday,is_weekday,sales_total
0,0,1,13.0,2013-01-01 00:00:00+00:00,1,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,0.84,35200264013,35200264013,2013,1,1,1,1,True,10.92
1,1,1,11.0,2013-01-02 00:00:00+00:00,2,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,0.84,35200264013,35200264013,2013,1,1,2,2,True,9.24
2,2,1,14.0,2013-01-03 00:00:00+00:00,3,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,0.84,35200264013,35200264013,2013,1,1,3,3,True,11.76
3,3,1,13.0,2013-01-04 00:00:00+00:00,4,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,0.84,35200264013,35200264013,2013,1,1,4,4,True,10.92
4,4,1,10.0,2013-01-05 00:00:00+00:00,5,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,0.84,35200264013,35200264013,2013,1,1,5,5,False,8.40


In [12]:
df = df.assign(sales_total=df.quantity*df.item_price)

In [13]:
df.head()

,Unnamed: 0,item_id,quantity,sale_date,sale_id,store_id,store_address,store_city,store_state,store_zipcode,...,item_price,item_upc12,item_upc14,year,quarter,month,day,weekday,is_weekday,sales_total
0,0,1,13.0,2013-01-01 00:00:00+00:00,1,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,0.84,35200264013,35200264013,2013,1,1,1,1,True,10.92
1,1,1,11.0,2013-01-02 00:00:00+00:00,2,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,0.84,35200264013,35200264013,2013,1,1,2,2,True,9.24
2,2,1,14.0,2013-01-03 00:00:00+00:00,3,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,0.84,35200264013,35200264013,2013,1,1,3,3,True,11.76
3,3,1,13.0,2013-01-04 00:00:00+00:00,4,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,0.84,35200264013,35200264013,2013,1,1,4,4,True,10.92
4,4,1,10.0,2013-01-05 00:00:00+00:00,5,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,0.84,35200264013,35200264013,2013,1,1,5,5,False,8.40


Create a new dataframe that aggregates the sales_total and sale_amount(item count) using sum and median by day of week.

In [29]:
df_new = df.groupby('weekday')[['sales_total', 'quantity']].agg(['sum', 'median'])
df_new

sales_total           quantity       
                  sum  median        sum median
weekday                                        
0        2.896118e+07  186.76  5385853.0   38.0
1        3.383954e+07  218.66  6293481.0   44.0
2        3.392832e+07  218.92  6312090.0   44.0
3        3.628727e+07  234.52  6749880.0   47.0
4        3.868773e+07  250.43  7198021.0   50.0
5        4.116177e+07  266.97  7655482.0   53.0
6        4.359486e+07  282.47  8109705.0   57.0

Explore the pandas DataFrame.diff() function. Create a new column that is the result of the current sales - the previous days sales.

In [24]:
df = df.assign(sales_diff=df.sales_total.diff(periods=-1))

In [25]:
df.head()

,Unnamed: 0,item_id,quantity,sale_date,sale_id,store_id,store_address,store_city,store_state,store_zipcode,...,item_upc12,item_upc14,year,quarter,month,day,weekday,is_weekday,sales_total,sales_diff
0,0,1,13.0,2013-01-01 00:00:00+00:00,1,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,35200264013,35200264013,2013,1,1,1,1,True,10.92,1.68
1,1,1,11.0,2013-01-02 00:00:00+00:00,2,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,35200264013,35200264013,2013,1,1,2,2,True,9.24,-2.52
2,2,1,14.0,2013-01-03 00:00:00+00:00,3,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,35200264013,35200264013,2013,1,1,3,3,True,11.76,0.84
3,3,1,13.0,2013-01-04 00:00:00+00:00,4,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,35200264013,35200264013,2013,1,1,4,4,True,10.92,2.52
4,4,1,10.0,2013-01-05 00:00:00+00:00,5,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,...,35200264013,35200264013,2013,1,1,5,5,False,8.40,-1.68


Write a function to set the index to be the datetime variable.

In [31]:
df.set_index('sale_date', inplace=True)